# Setup: Azure Storage & Event Hubs Connection

## Purpose
This notebook establishes connections to:
- **Azure Blob Storage** (for reading Stream Analytics output)
- **Event Hubs** (for real-time streaming ingestion)

## Prerequisites
- Storage account access key
- Event Hubs connection string

## Outputs
- Configured Spark session with storage access
- Validated Event Hubs connectivity

## 1. Azure Blob Storage Configuration

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Storage account credentials
# NOTE: In production, use Azure Key Vault or Databricks Secrets instead of hardcoding
storage_account_name = "YOUR_STORAGE_ACCOUNT_NAME"  # Replace with your Azure Storage account name
container_name = "eventhub-capture"
storage_account_key = "YOUR_KEY_HERE"  # TODO: Replace with dbutils.secrets.get()

# Configure Spark to access Azure Blob Storage
spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net",
    storage_account_key
)

print(f"✅ Storage configured: wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/")

## 2. Verify Storage Connection

In [ ]:
# Build storage path
storage_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/"

# Test connection by listing files
try:
    files = dbutils.fs.ls(storage_path)
    print(f"Connection successful! Found {len(files)} items in container.")
    display(dbutils.fs.ls(storage_path))
except Exception as e:
    print(f"Connection failed: {e}")
    raise

## 3. Event Hubs Configuration

**Connection String Format:**
```
Endpoint=sb://NAMESPACE.servicebus.windows.net/;
SharedAccessKeyName=KEY_NAME;
SharedAccessKey=KEY;
EntityPath=HUB_NAME
```

In [ ]:
# Event Hubs connection details
# SECURITY NOTE: Use Databricks secrets in production
CONNECTION_STRING = "YOUR_CONNECTION_STRING_HERE"  # TODO: Use dbutils.secrets

# Encrypt connection string for Spark
ehConf = {
  'eventhubs.connectionString': sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(CONNECTION_STRING)
}

print("✅ Event Hubs configuration ready")

## 4. Validation Summary

Run the cells above to verify:
- ✅ Storage account accessible
- ✅ Container files visible
- ✅ Event Hubs connection string configured

**Next Step:** Run `02_bronze_layer` to start ingesting data